# YOLOv11 Model Training
- Kaggle was used for training with T4 x2 GPUs

# CC0 1.0 Universal (Public Domain Dedication)
# Copyright (C) 2025 JustSplash8501
# This work is dedicated to the public domain under the CC0 1.0 Universal license.
# You can copy, modify, distribute, and perform the work, even for commercial purposes,
# without asking permission. No rights are reserved.
# Full license text: https://creativecommons.org/publicdomain/zero/1.0/legalcode.txt

# Unzip Dataset

In [ ]:
import zipfile

zip_path = "/content/Pistols.v1-resize-416x416.yolov11.zip"
extract_folder = "/content/extracted/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

%ls -lh /content/extracted/


total 16K
-rw-r--r-- 1 root root  253 Dec 21 05:04 data.yaml
drwxr-xr-x 4 root root 4.0K Dec 21 05:04 export
-rw-r--r-- 1 root root 1.2K Dec 21 05:04 README.dataset.txt
-rw-r--r-- 1 root root  980 Dec 21 05:04 README.roboflow.txt


# Split Dataset for YOLO Model

In [1]:
import random
from pathlib import Path
import shutil
import yaml

def prepare_yolo_dataset(images_dir, labels_dir, output_dir,
                         train_ratio=0.8, val_ratio=0.2,
                         seed=42, class_names=['pistol']):
    """
    Prepares a YOLOv8 dataset with train/val splits and proper folder structure.

    Dataset structure:
    dataset/
    ├── data.yaml
    ├── train/
    │   ├── images/
    │   └── labels/
    └── val/
        ├── images/
        └── labels/

    Args:
        images_dir: Folder containing all images
        labels_dir: Folder containing all .txt labels
        output_dir: Root output folder (dataset/)
        train_ratio, val_ratio: Split fractions (sum=1)
        seed: Random seed
        class_names: List of class names
    """
    assert abs(train_ratio + val_ratio - 1.0) < 1e-6, "Ratios must sum to 1"

    images_dir = Path(images_dir)
    labels_dir = Path(labels_dir)
    output_dir = Path(output_dir)
    random.seed(seed)

    # Create folder structure
    splits_dirs = {}
    for split in ["train", "val"]:
        splits_dirs[split] = {
            "images": output_dir / f"{split}/images",
            "labels": output_dir / f"{split}/labels"
        }
        for d in splits_dirs[split].values():
            d.mkdir(parents=True, exist_ok=True)

    # Get all label files
    label_files = list(labels_dir.glob("*.txt"))
    random.shuffle(label_files)
    n_total = len(label_files)
    n_train = int(train_ratio * n_total)
    # val takes remaining
    splits = {
        "train": label_files[:n_train],
        "val": label_files[n_train:]
    }

    # Copy labels and images into split folders
    for split_name, files in splits.items():
        for lbl in files:
            # Copy label
            shutil.copy(lbl, splits_dirs[split_name]["labels"] / lbl.name)
            # Copy corresponding image
            img_name = lbl.stem + ".jpg"
            img_file = images_dir / img_name
            if not img_file.exists():
                raise FileNotFoundError(f"Image not found: {img_file}")
            shutil.copy(img_file, splits_dirs[split_name]["images"] / img_name)

    # Create data.yaml
    data_yaml = {
        "train": "train/images",
        "val": "val/images",
        "nc": len(class_names),
        "names": class_names
    }

    yaml_path = output_dir / "data.yaml"
    with open(yaml_path, "w") as f:
        yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

    print(f"Dataset prepared at {output_dir}")
    print(f"Train: {len(splits['train'])} labels, Val: {len(splits['val'])}")
    print(f"data.yaml saved at: {yaml_path}")



# Restructure folders
prepare_yolo_dataset(
    images_dir="extracted/export/images",
    labels_dir="extracted/export/labels",
    output_dir="dataset",
    train_ratio=0.85,
    val_ratio=0.15,
    class_names=['pistol']
)


Dataset prepared at dataset
Train: 0 labels, Val: 0
data.yaml saved at: dataset/data.yaml


# Train and Save Model

In [ ]:
%pip install -q ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.6 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
from ultralytics import YOLO
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = YOLO("yolov10s.pt")

# Train the model
model.train(
    data="/kaggle/working/dataset/data.yaml",
    # CONFIG
    name="pistol_model_v1_0",
    pretrained=True,
    amp=True,
    half=True,
    val=True,
    workers=4,
    save_period=10,
    # AUGMENTATION
    mosaic=True,            # combines 4 images into one
    mixup=True,             # combines two images with labels
    hsv_h=0.015,            # hue
    hsv_s=0.7,              # saturation
    hsv_v=0.4,              # brightness/value
    degrees=10.0,           # rotation
    translate=0.1,          # shift
    scale=0.1,              # zoom in/out
    shear=2.0,              # shear transformation
    flipud=0.0,             # vertical flip probability
    fliplr=0.5,             # horizontal flip probability
    # PARAMS
    epochs=100,
    imgsz=640,
    batch=32,
    # CALLBACKS
    patience=10,
    time=3,
    cos_lr=True,
    device=[0, 1]
)

from pathlib import Path
import shutil
from IPython.display import FileLink

def download_detect_folder_as_zip(runs_dir='/kaggle/working/runs/detect',
                                   output_name='yolo_detect_runs.zip'):
    """Download entire detect folder as zip (works in Kaggle)"""
    
    runs_dir = Path(runs_dir)
    
    if not runs_dir.exists():
        raise FileNotFoundError(f"{runs_dir} does not exist")
    
    print(f"Zipping entire folder: {runs_dir}")
    
    # Create zip of the entire detect folder
    zip_path = shutil.make_archive(
        output_name.replace('.zip', ''),  # Remove .zip as make_archive adds it
        'zip',
        runs_dir.parent,  # Parent directory (runs/)
        runs_dir.name     # Folder to zip (detect/)
    )
    
    print(f"Zip created: {zip_path}")
    print("Click the link below to download:")
    
    # Create download link for Kaggle
    return FileLink(zip_path)

# Download entire detect folder
download_detect_folder_as_zip()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using device: cuda
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ker